In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer


In [74]:
salary_train = pd.read_csv('salary-train.csv')
salary_train['FullDescription'] = salary_train['FullDescription'].replace('[^a-zA-Z0-9]', " ", regex=True)
for i in salary_train['FullDescription']:
    i.lower()

In [76]:
# обучить замену текста по 5 самым повторяющимся словам в колне FullDescription
tfid = TfidfVectorizer( min_df=5)
tfid.fit(salary_train['FullDescription'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [77]:
X_train_text = tfid.transform(salary_train['FullDescription'])

In [78]:
# Заменить пусты значения в стоблцах на 'nan'
salary_train['LocationNormalized'].fillna('nan', inplace=True)
salary_train['ContractTime'].fillna('nan', inplace=True)


In [79]:
# Преобразховать текст в массив 
dictVect = DictVectorizer()
X_train_cat = dictVect.fit_transform((salary_train[['LocationNormalized', 'ContractTime']]).to_dict('record'))


In [80]:
from scipy.sparse import hstack

X_train = hstack([X_train_text, X_train_cat])

<60000x24627 sparse matrix of type '<class 'numpy.float64'>'
	with 8485759 stored elements in COOrdinate format>

In [81]:
y_train = salary_train["SalaryNormalized"]
ridge = Ridge(alpha=1, random_state=242)
ridge.fit(X_train, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=242, solver='auto', tol=0.001)

In [96]:
test_file = pd.read_csv('salary-test-mini.csv')
test_file

test_file['FullDescription'] =  test_file['FullDescription'].replace('[^a-zA-Z0-9]', " ", regex=True)
for i in test_file['FullDescription']:
    i.lower()
    
X_test_text = tfid.transform(test_file['FullDescription'])
X_test_cat = dictVect.transform((test_file[['LocationNormalized', 'ContractTime']]).to_dict('record'))

X_test = hstack([X_test_text, X_test_cat]) 
ridge.predict(X_test)

array([ 56588.0026 ,  37228.77631])